In [ ]:
#Importing the required headers
import requests
import json
import pandas as pd
import re
import csv
#Creating the session
session = requests.session()


In [ ]:
#Moving to store location page
main = session.get("https://www.lenskart.com/stores")
with open("demo.html",'w',encoding='utf-8') as f:
  f.write(main.text)

In [ ]:
#Using regex to find all the state names
pattern= re.compile(r'slug\"\:\"([^"\d]+)\"')
matches = pattern.findall(main.text)

In [ ]:
#Iterating thorough the states and storing in the list
store_location = []
for match in matches:
    store_location.append(match)
print(store_location)

['andhra-pradesh', 'andmaan-nicobar', 'arunachal-pradesh', 'assam', 'bihar', 'chandigarh', 'chennai', 'chhattisgarh', 'dadra-and-nagar-haveli', 'delhi', 'goa', 'gujarat', 'gwalior', 'haryana', 'himachal-pradesh', 'hyderabad', 'jammu-and-kashmir', 'jharkhand', 'karnataka', 'kerala', 'madhya-pradesh', 'maharashtra', 'manipur', 'nagaland', 'new-delhi', 'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim', 'tamil-nadu', 'tamil\xa0nadu', 'telangana', 'tripura', 'uttar-pradesh', 'uttarakhand', 'utter-pradesh', 'uttrakhand', 'west-bengal']


In [ ]:
#Getting the details of each state's store
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Xstoreaccesskey": "pNRq8BGvIHk/G9AwlBxKG5lz97eYDaxnWO0YYs+VrMY="
}

payload = {
    "keyword": None,
    "range": 10,
    "perPage": 9999
}


In [ ]:
#Creating a separate json file for each state
for item in store_location:
  payload["keyword"] = item
  res = session.post(r'https://locator-stores.lenskart.com/api/v3/store/list', headers=header, data=payload)
  filename = item + '.json'
  with open(filename, 'w') as json_file:
        json.dump(res.json(), json_file)

In [ ]:
fieldnames = ['id', 'name','status', 'address', 'city','state','phone', 'email',
              'openinghr', 'closinghr', 'Rating', 'Reviews']

# Initialize an empty list to store data rows
data_rows = []

# Iterate over each store location
for item in store_location:
    with open(f'{item}.json', 'r') as fr:
        # Read JSON data
        json_data = json.load(fr)
        print(json_data)

        # Extract relevant data from JSON
        for store in json_data['data']['data']:
            store_data = {
                'id': store['id'],
                'name': store['alt_store_name_catch'],
                'status':store['status'],
                'address': store['address_full'],
                'city':store['address_city'],
                'state': store['address_state'],
                'phone': store['store_phone'],
                'email': store['store_email'],
                'openinghr': store['business_hours_openi'],
                'closinghr': store['business_hours_closi'],
                'Rating': store['ratingAndReviewCount']['averageRating'],
                'Reviews': store['ratingAndReviewCount']['totalReviews']
            }
            # Append store data to the list
            data_rows.append(store_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#creating a csv file and loading data to it
with open('store.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write data rows
    for row in data_rows:
        writer.writerow(row)

print("CSV file created successfully.")

CSV file created successfully.
